# Feeding data to a model 
    
 - Queue runners
 - Dataset(TFRecord)

### File I/O using queue runners 

 - Dataset Class는 Image read 기능을 직접적으로 제공하지 않기 때문에, 하위 레벨 API 인 `queue runner`를 이용한다. 
 - queue runners 는 일종의 thread runner 
 - Loading 과 Session이 병렬적으로 동작한다. $\rightarrow$ Loading Delay없이 연속적인 training이 가능하다. 
 - Directory 내 모든 학습 데이터(eg. Images)들을 직접 load하여 train할 경우 유용!
 

#### $\sharp$ 데이터를 학습 시키는 두 가지 방법 

 1 . Load $\rightarrow$ Preprocessing $\rightarrow$ train
     > raw data를 로드하여 전처리를 수행 한 후 학습 
 
 2 . Load $\rightarrow$ Preprocessing $\rightarrow$ write datasets $\rightarrow$ load dataset $\rightarrow$ train
     > raw data를 미리 전처리하여 파일로 쓰고, 전처리된 데이터를 로드하여 학습

> - 전처리 라인 길면, 학습의 성능이 저하 될 수 있다. 
> - 일반적으로는 전처리 파이프라인을 별도로 운영하고, 전처리된 데이터를 활용하지만, Case by Case이다. 
> - 각 데이터 포맷 별 로 to tfrecord transformer를 구축하여 트레인 시스템 운영해보겠다. 


### File I/O using Dataset(TFRecord)

- Tensorflow에서 권장하는 record dataset 
- csv와 유사하게 Dataset 클래스로 쉽게 데이터를 로드 할 수 있다. 
- 실제로 file load는 한 번만 이루어지므로 효율적인 data read각 가능하다. 
- HDFS 활용 시 매우 효과적이다. 

## Implementation

In [4]:
import tensorflow as tf
import matplotlib.pyplot as plt 
import numpy as np 
import os 

tf.reset_default_graph()

image_dir = "./cnn_dataset/images"
age_dir = './cnn_dataset/age.csv'
# Dataset 디렉토리에 있는 파일들의 리스트를 저장 
imgname_list = [os.path.join(image_dir, name) for name in os.listdir(image_dir)]

imgname_queue = tf.train.string_input_producer(imgname_list, num_epochs=1, shuffle=False,)
agename_queue = tf.train.string_input_producer([age_dir], num_epochs=1,shuffle=False)

img_reader = tf.WholeFileReader()
age_reader = tf.TextLineReader()

# key_img:파일이름, values_img:데이터  return 
key_img, raw_img = img_reader.read(imgname_queue)
key_age, raw_age = age_reader.read(agename_queue)

decoded_img = tf.image.decode_png(raw_img)
decoded_age = tf.decode_csv(raw_age, [[0]])

# gray image reshaping 
mean_reduced_image = tf.reduce_mean(decoded_img, axis=-1)
reshaped_img = tf.reshape(mean_reduced_image, [-1])

In [5]:
with tf.Session() as sess :
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess, coord)

#     sess.run(tf.global_variables_initializer())
#     tf.initialize_all_variables()
    
    face_train_dir = './cnn_dataset/face_train.tfrecord'
    face_test_dir = './cnn_dataset/face_test.tfrecord'
    
    train_writer = tf.python_io.TFRecordWriter(face_train_dir)
    test_writer = tf.python_io.TFRecordWriter(face_test_dir)
    
    for i in range(99999999):
        try :
            _age, _img, _key = sess.run([decoded_age, reshaped_img, key_img])
            
            example = tf.train.Example()
            example.features.feature['age'].int64_list.value.append(_age[0])
            example.features.feature['img'].int64_list.value.extend(_img.tolist())
            
            if i < 6000:
                train_writer.write(example.SerializeToString())
                if i % 500:
                    print('{} train data has been written'.format(i))
            else:
                test_writer.write(example.SerializeToString())
                if i % 500 == 0 :
                    print('{} test data has been written'.format(i))
        
        except tf.errors.OutOfRangeError:
            print('size of total dataset {}'.format(i))
            break
    
    train_writer.close()
    test_writer.close()
    coord.request_stop()
    coord.join(thread)

1 train data has been written
2 train data has been written
3 train data has been written
4 train data has been written
5 train data has been written
6 train data has been written
7 train data has been written
8 train data has been written
9 train data has been written
10 train data has been written
11 train data has been written
12 train data has been written
13 train data has been written
14 train data has been written
15 train data has been written
16 train data has been written
17 train data has been written
18 train data has been written
19 train data has been written
20 train data has been written
21 train data has been written
22 train data has been written
23 train data has been written
24 train data has been written
25 train data has been written
26 train data has been written
27 train data has been written
28 train data has been written
29 train data has been written
30 train data has been written
31 train data has been written
32 train data has been written
33 train data has

419 train data has been written
420 train data has been written
421 train data has been written
422 train data has been written
423 train data has been written
424 train data has been written
425 train data has been written
426 train data has been written
427 train data has been written
428 train data has been written
429 train data has been written
430 train data has been written
431 train data has been written
432 train data has been written
433 train data has been written
434 train data has been written
435 train data has been written
436 train data has been written
437 train data has been written
438 train data has been written
439 train data has been written
440 train data has been written
441 train data has been written
442 train data has been written
443 train data has been written
444 train data has been written
445 train data has been written
446 train data has been written
447 train data has been written
448 train data has been written
449 train data has been written
450 trai

827 train data has been written
828 train data has been written
829 train data has been written
830 train data has been written
831 train data has been written
832 train data has been written
833 train data has been written
834 train data has been written
835 train data has been written
836 train data has been written
837 train data has been written
838 train data has been written
839 train data has been written
840 train data has been written
841 train data has been written
842 train data has been written
843 train data has been written
844 train data has been written
845 train data has been written
846 train data has been written
847 train data has been written
848 train data has been written
849 train data has been written
850 train data has been written
851 train data has been written
852 train data has been written
853 train data has been written
854 train data has been written
855 train data has been written
856 train data has been written
857 train data has been written
858 trai

1183 train data has been written
1184 train data has been written
1185 train data has been written
1186 train data has been written
1187 train data has been written
1188 train data has been written
1189 train data has been written
1190 train data has been written
1191 train data has been written
1192 train data has been written
1193 train data has been written
1194 train data has been written
1195 train data has been written
1196 train data has been written
1197 train data has been written
1198 train data has been written
1199 train data has been written
1200 train data has been written
1201 train data has been written
1202 train data has been written
1203 train data has been written
1204 train data has been written
1205 train data has been written
1206 train data has been written
1207 train data has been written
1208 train data has been written
1209 train data has been written
1210 train data has been written
1211 train data has been written
1212 train data has been written
1213 train

1569 train data has been written
1570 train data has been written
1571 train data has been written
1572 train data has been written
1573 train data has been written
1574 train data has been written
1575 train data has been written
1576 train data has been written
1577 train data has been written
1578 train data has been written
1579 train data has been written
1580 train data has been written
1581 train data has been written
1582 train data has been written
1583 train data has been written
1584 train data has been written
1585 train data has been written
1586 train data has been written
1587 train data has been written
1588 train data has been written
1589 train data has been written
1590 train data has been written
1591 train data has been written
1592 train data has been written
1593 train data has been written
1594 train data has been written
1595 train data has been written
1596 train data has been written
1597 train data has been written
1598 train data has been written
1599 train

...

4249 train data has been written
4250 train data has been written
4251 train data has been written
4252 train data has been written
4253 train data has been written
4254 train data has been written
4255 train data has been written
4256 train data has been written
4257 train data has been written
4258 train data has been written
4259 train data has been written
4260 train data has been written
4261 train data has been written
4262 train data has been written
4263 train data has been written
4264 train data has been written
4265 train data has been written
4266 train data has been written
4267 train data has been written
4268 train data has been written
4269 train data has been written
4270 train data has been written
4271 train data has been written
4272 train data has been written
4273 train data has been written
4274 train data has been written
4275 train data has been written
4276 train data has been written
4277 train data has been written
4278 train data has been written
4279 train

4664 train data has been written
4665 train data has been written
4666 train data has been written
4667 train data has been written
4668 train data has been written
4669 train data has been written
4670 train data has been written
4671 train data has been written
4672 train data has been written
4673 train data has been written
4674 train data has been written
4675 train data has been written
4676 train data has been written
4677 train data has been written
4678 train data has been written
4679 train data has been written
4680 train data has been written
4681 train data has been written
4682 train data has been written
4683 train data has been written
4684 train data has been written
4685 train data has been written
4686 train data has been written
4687 train data has been written
4688 train data has been written
4689 train data has been written
4690 train data has been written
4691 train data has been written
4692 train data has been written
4693 train data has been written
4694 train

5075 train data has been written
5076 train data has been written
5077 train data has been written
5078 train data has been written
5079 train data has been written
5080 train data has been written
5081 train data has been written
5082 train data has been written
5083 train data has been written
5084 train data has been written
5085 train data has been written
5086 train data has been written
5087 train data has been written
5088 train data has been written
5089 train data has been written
5090 train data has been written
5091 train data has been written
5092 train data has been written
5093 train data has been written
5094 train data has been written
5095 train data has been written
5096 train data has been written
5097 train data has been written
5098 train data has been written
5099 train data has been written
5100 train data has been written
5101 train data has been written
5102 train data has been written
5103 train data has been written
5104 train data has been written
5105 train

5473 train data has been written
5474 train data has been written
5475 train data has been written
5476 train data has been written
5477 train data has been written
5478 train data has been written
5479 train data has been written
5480 train data has been written
5481 train data has been written
5482 train data has been written
5483 train data has been written
5484 train data has been written
5485 train data has been written
5486 train data has been written
5487 train data has been written
5488 train data has been written
5489 train data has been written
5490 train data has been written
5491 train data has been written
5492 train data has been written
5493 train data has been written
5494 train data has been written
5495 train data has been written
5496 train data has been written
5497 train data has been written
5498 train data has been written
5499 train data has been written
5501 train data has been written
5502 train data has been written
5503 train data has been written
5504 train

5881 train data has been written
5882 train data has been written
5883 train data has been written
5884 train data has been written
5885 train data has been written
5886 train data has been written
5887 train data has been written
5888 train data has been written
5889 train data has been written
5890 train data has been written
5891 train data has been written
5892 train data has been written
5893 train data has been written
5894 train data has been written
5895 train data has been written
5896 train data has been written
5897 train data has been written
5898 train data has been written
5899 train data has been written
5900 train data has been written
5901 train data has been written
5902 train data has been written
5903 train data has been written
5904 train data has been written
5905 train data has been written
5906 train data has been written
5907 train data has been written
5908 train data has been written
5909 train data has been written
5910 train data has been written
5911 train